In [6]:
import numpy as np, pandas as pd
import glob
import os 
import json
import sys
import numpy
from math import log, log10
from gdpyc import GasMap, DustMap
from astropy.coordinates import SkyCoord
from astropy import units as u

df = pd.read_csv('./FGL_11152023_GammaCan.csv')

df = df[df['FGL'].str[-1]!='c']

df['GLON'] = SkyCoord(ra=df['ra'].values*u.degree, dec=df['dec'].values*u.degree, frame='icrs').galactic.l.degree
df['GLAT'] = SkyCoord(ra=df['ra'].values*u.degree, dec=df['dec'].values*u.degree, frame='icrs').galactic.b.degree

df = df.sort_values(by=['flux_aper90_ave_b'], ascending=False).reset_index(drop=True)

# print(df.columns)

df[['FGL','X_name','Class','name','flux_aper90_ave_b']]

df['name_nospace'] = df['X_name'].str.replace(' ','')#apply(lambda x: x.replace(' ', ''))



df_alls = pd.read_csv('./FGL_11152023_GammaCan_alls.csv')

# light curve >50 counts
# spectrum > 100 counts

df_select3 = df[df['X_name'].isin(df_alls.loc[df_alls['src_cnts_aper90_b']>=50, 'name'].unique())].reset_index(drop=True)

df_select3 = df_select3[~df_select3['X_name'].isin(['2CXO J174457.8-290509', '2CXO J174507.0-290357','2CXO J174506.8-290537','2CXO J111459.1-611707','2CXO J150234.5+015205','2CXO J073717.0+653557'])].reset_index(drop=True)
# print(df_select3)

coords = SkyCoord(df_select3['ra'], df_select3['dec'], unit='deg')
df_select3['ebv'] = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
df_select3['nH_from_AV'] = 2.21 * 3.1 * df_select3['ebv'] * 0.1
df_select3['nH']  = np.log10(GasMap.nh(coords, nhmap='LAB').value) #/ 1e22 # nH in unit of 1.e22 atoms /cm2

print(df_select3[['FGL','X_name','Class','CT','name','name_nospace','ra', 'dec','flux_aper90_ave_b','var_intra_prob',
       'var_inter_prob','nH']])

             FGL                 X_name Class          CT  \
0   J0859.3-4342  2CXO J085927.0-434528    CV    2.451807   
1   J1742.5-2833  2CXO J174237.6-283726    NS    5.113294   
2   J0335.6-0727  2CXO J033532.7-072741   AGN    2.304350   
3   J1032.0+5725  2CXO J103143.2+573252   AGN   30.824433   
4   J1843.7-3227  2CXO J184316.0-322414    NS    2.753150   
5   J1256.9+2736  2CXO J125740.2+273118   AGN   56.039205   
6   J1435.4+3338  2CXO J143542.6+333404   AGN  427.564247   
7   J0639.1-8009  2CXO J063554.8-800814   AGN   31.027193   
8   J0859.3-4342  2CXO J085926.8-434933   AGN    2.174129   
9   J1502.6+0207  2CXO J150237.4+015813   AGN   73.879141   
10  J1032.0+5725  2CXO J103202.9+573208   AGN    7.280492   
11  J1435.4+3338  2CXO J143513.2+333118   AGN    8.646924   
12  J0639.1-8009  2CXO J063719.7-801230   AGN   10.428117   
13  J0737.4+6535  2CXO J073733.4+653307   AGN    2.295048   
14  J1843.7-3227  2CXO J184337.6-322514   AGN    4.848028   
15  J1256.9+2736  2CXO J

In [3]:
df.columns

Index(['P_AGN', 'P_CV', 'P_HM-STAR', 'P_HMXB', 'P_LM-STAR', 'P_LMXB', 'P_NS',
       'P_YSO', 'e_P_AGN', 'e_P_CV', 'e_P_HM-STAR', 'e_P_HMXB', 'e_P_LM-STAR',
       'e_P_LMXB', 'e_P_NS', 'e_P_YSO', 'Class', 'Class_prob', 'Class_prob_e',
       'name', 'CT', 'X_name', 'FGL', 'ra', 'dec', 'r0', 'significance',
       'Fcsc_s', 'e_Fcsc_s', 'Fcsc_m', 'e_Fcsc_m', 'Fcsc_h', 'e_Fcsc_h',
       'flux_aper90_ave_b', 'e_flux_aper90_ave_b', 'var_intra_prob',
       'var_inter_prob', 'MW_MW', 'MW_counts', 'match_flag', 'p_any', 'p_i',
       'Gmag', 'BPmag', 'RPmag', 'e_Gmag', 'e_BPmag', 'e_RPmag', 'RPlx', 'PM',
       'epsi', 'sepsi', 'ruwe', 'rgeo', 'Jmag', 'Hmag', 'Kmag', 'e_Jmag',
       'e_Hmag', 'e_Kmag', 'W1mag', 'W2mag', 'W3mag', 'e_W1mag', 'e_W2mag',
       'e_W3mag', 'r_hat', 'X_PA', 'G_ra', 'G_dec', 'G_aF', 'G_bF', 'G_PA',
       'sep', 'Fb', 'HR_hms', 'popup', 'shape', 'color', 'name_nospace'],
      dtype='object')

In [7]:
df_srcs = pd.DataFrame()


for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    # if src != '2CXOJ174457.8-290509':
    #     continue
    #print(src)
    # if index >40:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    
    src_stat = {'FGL':df_src['FGL'], 'name': src_short,'l':df_src['GLON'],'b':df_src['GLAT'], 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH'], 'var_inter':df_src['var_inter_prob'], 'var_intra':df_src['var_intra_prob']}
    # coords = SkyCoord(ra, dec, unit='deg')
    # ebv = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
    # nH_from_AV = df_src['nH_from_AV']# 2.21 * 3.1 * ebv * 0.1
    # nH  = df_src['nH']# GasMap.nh(coords, nhmap='LAB').value / 1e22 # nH in unit of 1.e22 atoms /cm2
    #print(f'nH: {nH} 1.e22 atoms /cm2')

    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']

    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')

    dir_abs = f'/home/orion51/Desktop/Research/spectral_fitting/FGL_spectral_fitting/Individual_sources/{src}/'

    
    os.system(f'python model_compare.py {dir_abs}powerlaw_out_/ {dir_abs}mekal_out_/ {dir_abs}bb_out_/  > {src}_model_compare.txt')

    prefixes = [f'{dir_abs}powerlaw_out_/',  f'{dir_abs}mekal_out_/', f'{dir_abs}bb_out_/']
    models = dict([(f, json.load(open(f + "info/results.json"))['logz']) for f in prefixes])

    best = max(models, key=models.__getitem__)
    Zbest = models[best]
    for m in models: models[m] -= Zbest
    Ztotal = log(sum(numpy.exp([Z for Z in list(models.values())])))
    limit = 30 # for example, Jeffreys scale for the Bayes factor

    # print()
    # print('Model comparison')
    # print('****************')
    # print()



    for m, md, md_f,p_n,p_l_n,p_u_n in zip(models, ['pl', 'mk', 'bb'], ['powerlaw','mekal','bb'], ['pl1.PhoIndex_median','src1.logkT_median','src1.logkT_median'], ['pl1.PhoIndex_errlo','src1.logkT_errlo','src1.logkT_errlo'], ['pl1.PhoIndex_errup','src1.logkT_errup','src1.logkT_errup']):
        df_src_sum = pd.read_csv(f'{dir_abs}{md_f}_out_/info/post_summary.csv')
        # print(df_src_sum.columns)
        src_stat[md],src_stat[md+'_l'],src_stat[md+'_u'] = df_src_sum.loc[0, p_n],df_src_sum.loc[0, p_l_n],df_src_sum.loc[0, p_u_n]
        Zrel = models[m]
        src_stat['log10Z_'+md] = Zrel / log(10)
        if Zrel == 0:
            src_stat['nH'],src_stat['nH_l'],src_stat['nH_u'] = df_src_sum.loc[0, 'src1.lognH_median'],df_src_sum.loc[0, 'src1.lognH_errlo'],df_src_sum.loc[0, 'src1.lognH_errup']
    df_sr = pd.DataFrame(data=src_stat, index=[0])
    df_srcs = pd.concat([df_srcs, df_sr], ignore_index=True, sort=False)
    # for md in ['powerlaw','mekal','bb']:

    print('  <tr>')
    print(f"   <td>{src_stat['name']}</td>")
    print(f"   <td>{src_stat['FGL']}</td>")
    print(f"   <td>{src_stat['l']:.7f}</td>")
    print(f"   <td>{src_stat['b']:.7f}</td>")
    print(f"   <td>{src_stat['Class']}</td>")
    print(f"   <td>{src_stat['CT']:.2f}</td>")
    print(f"   <td>{src_stat['fb']:.2e}</td>")
    print(f"   <td>{src_stat['var_inter']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_lc_500s.png" target="_blank">{src_stat["var_intra"]:.2f}</a></td>')
    print(f"   <td>{src_stat['nobs']:.1f}</td>")
    print(f"   <td>{src_stat['ncounts']:.1f}</td>")
    print(f"   <td>{src_stat['pl']:.2f}<sup>+{src_stat['pl_u']-src_stat['pl']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['pl']-src_stat['pl_l']:.2}</sub></td>")
    # print(f"   <td>{src_stat['log10Z_pl']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["log10Z_pl"]:.2f}</a> (<a class="image-link" href="./images/{src}_pl_corner.pdf" target="_blank">covar</a>)</td>')
    print(f"   <td>{10**src_stat['mk']:.2f}<sup>+{10**src_stat['mk_u']-10**src_stat['mk']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{10**src_stat['mk']-10**src_stat['mk_l']:.2}</sub></td>")
    print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{src_stat["log10Z_mk"]:.2f}</a> (<a class="image-link" href="./images/{src}_mk_corner.pdf" target="_blank">covar</a>)</td>')
    # print(f"   <td>{src_stat['log10Z_mk']:.2f}</td>")
    print(f"   <td>{10**src_stat['bb']:.2f}<sup>+{10**src_stat['bb_u']-10**src_stat['bb']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{10**src_stat['bb']-10**src_stat['bb_l']:.2}</sub></td>")
    # print(f"   <td>{src_stat['log10Z_bb']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{src_stat["log10Z_bb"]:.2f}</a> (<a class="image-link" href="./images/{src}_bb_corner.pdf" target="_blank">covar</a>)</td>')
    print(f"   <td>{src_stat['nH']:.2f}<sup>+{src_stat['nH_u']-src_stat['nH']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['nH']-src_stat['nH_l']:.2}</sub></td>")
    print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
    print('  </tr>')

  <tr>
   <td>J085927.0-434528</td>
   <td>J0859.3-4342</td>
   <td>265.1473173</td>
   <td>1.4499867</td>
   <td>CV</td>
   <td>2.45</td>
   <td>1.59e-12</td>
   <td>nan</td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_lc_500s.png" target="_blank">1.00</a></td>
   <td>1.0</td>
   <td>6082.5</td>
   <td>1.63<sup>+0.06</sup><sub style='position: relative; right: +2.3em;'>-0.054</sub></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_pl_spectrum.png" target="_blank">-10.45</a> (<a class="image-link" href="./images/2CXOJ085927.0-434528_pl_corner.pdf" target="_blank">covar</a>)</td>
   <td>7.82<sup>+0.81</sup><sub style='position: relative; right: +2.3em;'>-0.66</sub></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_mk_spectrum.png" target="_blank">0.00</a> (<a class="image-link" href="./images/2CXOJ085927.0-434528_mk_corner.pdf" target="_blank">covar</a>)</td>
   <td>1.18<sup>+0.02</sup><sub style='position: relative; right: +2

In [6]:
df_srcs.to_csv('FGL_11152023_GammaCan_summary.csv',index=False)